In [ ]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as mpl
from datetime import timedelta

from pylab import rcParams
rcParams['figure.figsize'] = 15, 10

import plotly.graph_objects as go
from plotly.subplots import make_subplots

colors = ["#e6194b", "#3cb44b", "#ffe119", "#0082c8", "#f58231", "#911eb4", "#46f0f0", "#f032e6", "#d2f53c", "#fabebe", "#008080", 
          "#e6beff", "#aa6e28", "#fffac8", "#800000", "#aaffc3", "#808000", "#ffd8b1", "#000080", "#808080", "#ffffff", "#000000"]

In [ ]:
actions_suffix = '_actions.csv'
value_suffix = '_state.csv'

In [ ]:
def get_dfs(stat_files):
    dfs = []
    for fname in stat_files:
        portfolio_prefix = f'../results/ce/{fname}/all'

        df = pd.read_csv(f"{portfolio_prefix}{value_suffix}")  # , index_col=0
        df = df.rename(columns={df.columns[0]: 'date'})
        df.loc[0, 'date'] = pd.to_datetime(df.loc[1]['date']) - timedelta(days=1)
        df['date'] = pd.to_datetime(df['date'])
        df['return'] = df['account_value'].pct_change() + 1
        df['equity'] = (df['return']).cumprod()
        df.loc[0, 'return'] = 1
        df.loc[0, 'equity'] = 1
        df.index = df['date']
        df = df.drop('date', axis=1)

        # df2 = pd.read_csv(f"{portfolio_prefix}{actions_suffix}", index_col=0)
        # df2.index = pd.to_datetime(df2.index)

        df['strategy'] = df['return']
        df_strategy = df[['strategy']]         
        df_returns = df_strategy.cumprod()
        dfs.append(df_returns)
    return dfs

def get_benchmark(dfs, mpt_only=False):
    benchmark_returns_1d = '../datasets/thesis/benchmark/1h_benchmark_returns.csv'
    df_benchmark = pd.read_csv(f"{benchmark_returns_1d}")
    df_benchmark.index = pd.to_datetime(df_benchmark['date'])
    df_benchmark = df_benchmark.drop('date', axis=1)
    if mpt_only:
        df_benchmark = df_benchmark.drop('BAH', axis=1)
        df_benchmark = df_benchmark.drop('CRP', axis=1)
    df_benchmark = df_benchmark[(df_benchmark.index >= dfs[0].index[0]) & (df_benchmark.index <= dfs[0].index[-1])]
    df_benchmark = df_benchmark.cumprod()
    return df_benchmark

def get_prices(sample_strat):
    stock_prices = '../datasets/thesis/1h/all_1h.csv'
    df3 = pd.read_csv(stock_prices)
    df3['date'] = pd.to_datetime(df3['date'])
    # get or column with close price for each stock per tick
    df3 = df3.pivot(index="date", columns="tic", values="close")
    # limit dataset to portfolio range
    df3 = df3[(df3.index >= sample_strat.index[0]) & (df3.index <= sample_strat.index[-1])]
    # get relative change from price
    df3 = df3.pct_change()
    # set first row to 0
    df3.loc[df3.iloc[0].name] = 0
    # get cumulative return for each stock
    df3 = (df3 + 1).cumprod()
    return df3

def plot_performance(title, names, strats, benchmarks, prices, plot_folder, show_prices=True, save=True):    
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    if show_prices:
        for idx, name in enumerate(prices.columns):
            fig.add_trace(go.Scatter(x=prices.index,y=prices[name],name=name, marker={'color':'#fff'}),secondary_y=False)

    for idx, name in enumerate(names):
        cdf = strats[idx]
        fig.add_trace(go.Scatter(x=cdf.index,y=cdf['strategy'],name=name),secondary_y=False)

    # benchmarks = dfs[0][["CRP", "BAH", "MPT"]]
    cs = ['#36454F', '#A9A9A9', '#8A9A5B']
    for idx, name in enumerate(benchmarks.columns):
        fig.add_trace(go.Scatter(x=benchmarks.index,y=benchmarks[name],name=name, marker={'color':cs[idx]}),secondary_y=False)

    fig.update_layout(title_text=f'{title} Strategy Performance compared to benchmarks', height=600)
    fig.show()
    if save:
        fig.write_html(f"{plot_folder}/performance_compare_{title.replace(' ', '_')}.html")

In [ ]:
names = ["run1", "run2", "run3", "run4-x6"]
stat_files = ["ce_1d_2301132037_1", "ce_1d_2301141639_1", "ce_1d_2301141954_1", "ce_1d_2301150052_6x"]

dfs = get_dfs(stat_files)
df_benchmark = get_benchmark(dfs, mpt_only=True)
df_prices = get_prices(dfs[0])

plot_folder = f'../results/ce/'
title = "1D Ensemble"
plot_performance(title, names, dfs, df_benchmark, df_prices, plot_folder, save=True, show_prices=False)

In [ ]:
names = ["run1", "run2", "run3", "run4-x6"]
stat_files = ["ce_12h_2301132144_1", "ce_12h_2301142119_1", "ce_12h_2301142318_1", "ce_12h_2301151203_6x"]

dfs = get_dfs(stat_files)
df_benchmark = get_benchmark(dfs, mpt_only=True)
df_prices = get_prices(dfs[0])

plot_folder = f'../results/ce/'
title = "12H Ensemble"
plot_performance(title, names, dfs, df_benchmark, df_prices, plot_folder, save=True, show_prices=False)

In [ ]:
names = ["run1", "run2", "run3", "run4-x6"]
stat_files = ["ce_6h_2301161759_1", "ce_6h_2301151826_1", "ce_6h_2301132259_1", "ce_6h_2301140113_6x"]

dfs = get_dfs(stat_files)
df_benchmark = get_benchmark(dfs, mpt_only=True)
df_prices = get_prices(dfs[0])

plot_folder = f'../results/ce/'
title = "6H Ensemble"
plot_performance(title, names, dfs, df_benchmark, df_prices, plot_folder, save=True, show_prices=False)

In [ ]:
names = ["run1", "run2", "run3", "run4-x6"]
stat_files = ["ce_1h_2301141010_1", "ce_1h_2301141238_1", "ce_1h_2301141443_1", "ce_1h_2301152309_6x"]

dfs = get_dfs(stat_files)
df_benchmark = get_benchmark(dfs, mpt_only=True)
df_prices = get_prices(dfs[0])

plot_folder = f'../results/ce/'
title = "1H Ensemble"
plot_performance(title, names, dfs, df_benchmark, df_prices, plot_folder, save=True, show_prices=False)

In [ ]:
cfg = {
    "PPO": ["EVAL_PPO_V208_01121518_1H_200M", "EVAL_PPO_V208_01121514_6H_550M", "EVAL_PPO_V208_01121435_12H_600M", "EVAL_PPO_V208_01121911_1D_180M"]
}

names = ["1H", "6H", "12H", "1D"]
dfs = get_dfs(names, "PPO", cfg["PPO"])


df_benchmark = get_benchmark(dfs)
df_prices = get_prices(dfs[0])

plot_folder = f'../results/cs_eval/'
model = "All PPO"
plot_performance(model, names, dfs, df_benchmark, df_prices, plot_folder, show_prices=False, save=True)

In [ ]:
cfg = {
    "TD3": ["EVAL_TD3_V214_01122319_1H_42M", "EVAL_TD3_V214_01130933_6H_54M", "EVAL_TD3_V214_01130958_12H_54M", "EVAL_TD3_V214_12201313_12M"]
}

names = ["1H", "6H", "12H", "1D"]
dfs = get_dfs(names, "TD3", cfg["TD3"])


df_benchmark = get_benchmark(dfs)
df_prices = get_prices(dfs[0])

plot_folder = f'../results/cs_eval/'
model = "All TD3"
plot_performance(model, names, dfs, df_benchmark, df_prices, plot_folder, show_prices=False, save=True)